In [1]:
import utils.buffers as buffers
import utils.stream as stream
import importlib
import pickle
import os
from math import ceil, log2
import utils.invert_index as invert_index
import numpy as np
import utils.query_runner as query_runner

In [2]:
# tamaño máximo para un buffer...
MAX_DICT_SIZE = 1000
# stream lee de listas de tamaño...
LIMIT = 400
# almacenar outputs temporales en...
main_path = "D:/Documents-D/temp/" 
dir1 = main_path + "dir1/"
dir2 = main_path + "dir2/"
# ubicación final del índice invertido...
index_filename = "index.pikl"
final = main_path + "inverted_index/"

In [11]:
importlib.reload(stream)
importlib.reload(invert_index)
importlib.reload(buffers)

<module 'utils.buffers' from 'c:\\Users\\Windows 11\\Documents\\UTEC\\Testing\\index_merge\\utils\\buffers.py'>

In [3]:
with open("stoplist.txt") as file:
    stoplist = [line.rstrip().lower() for line in file]

# test files
files = os.listdir("lyrics/")
files = ["lyrics/" + file for file in files]
doc_count = len(files)

# generate stream
ss = stream.Stream(files, stoplist, LIMIT)

Document lyrics/4_out_of_5.txt has been preprocessed!
Document lyrics/505.txt has been preprocessed!
Document lyrics/arabella.txt has been preprocessed!
Document lyrics/batphone.txt has been preprocessed!
Document lyrics/cornerstone.txt has been preprocessed!
Document lyrics/dance_little_liar.txt has been preprocessed!
Document lyrics/no1_party_anthem.txt has been preprocessed!
Document lyrics/one_for_the_road.txt has been preprocessed!
Document lyrics/when_the_sun_goes_down.txt has been preprocessed!


### **Construcción del índice**

In [4]:
# SPIMIndexConstruction
files_to_merge = []
n = 0
while not ss.empty:
    fn = invert_index.spimi_invert(dir2, ss, n, MAX_DICT_SIZE)
    files_to_merge.append(fn)
    n += 1
ss.end()
print(files_to_merge)

['D:/Documents-D/temp/dir2/f0.pkl', 'D:/Documents-D/temp/dir2/f1.pkl', 'D:/Documents-D/temp/dir2/f2.pkl', 'D:/Documents-D/temp/dir2/f3.pkl', 'D:/Documents-D/temp/dir2/f4.pkl', 'D:/Documents-D/temp/dir2/f5.pkl', 'D:/Documents-D/temp/dir2/f6.pkl', 'D:/Documents-D/temp/dir2/f7.pkl', 'D:/Documents-D/temp/dir2/f8.pkl', 'D:/Documents-D/temp/dir2/f9.pkl', 'D:/Documents-D/temp/dir2/f10.pkl', 'D:/Documents-D/temp/dir2/f11.pkl', 'D:/Documents-D/temp/dir2/f12.pkl', 'D:/Documents-D/temp/dir2/f13.pkl', 'D:/Documents-D/temp/dir2/f14.pkl', 'D:/Documents-D/temp/dir2/f15.pkl', 'D:/Documents-D/temp/dir2/f16.pkl', 'D:/Documents-D/temp/dir2/f17.pkl', 'D:/Documents-D/temp/dir2/f18.pkl', 'D:/Documents-D/temp/dir2/f19.pkl', 'D:/Documents-D/temp/dir2/f20.pkl', 'D:/Documents-D/temp/dir2/f21.pkl', 'D:/Documents-D/temp/dir2/f22.pkl', 'D:/Documents-D/temp/dir2/f23.pkl', 'D:/Documents-D/temp/dir2/f24.pkl']


In [ ]:
# Merge
levels = ceil(log2(n))
for i in range(levels):
    last_file = 0 # index of last file
    dir1, dir2 = dir2, dir1
    for j in range(ceil(n / pow(2, i + 1))):
        group_size = pow(2, i)
        wbuffer = buffers.WriteBuffer(dir2, last_file, last_file + 2 * group_size, n, MAX_DICT_SIZE)
        h1 = buffers.ReadBuffer(dir1, last_file, last_file + group_size, n)
        last_file += group_size
        h2 = buffers.ReadBuffer(dir1, last_file, last_file + group_size, n)
        last_file += group_size

        while not h1.over and not h2.over:
            h = h1
            if h2.word < h1.word or (h1.word == h2.word and h2.docID <= h1.docID):
                h = h2
            elem = h.get_next_pair()
            if i == ceil(log2(n)) -1:
                print(elem)
            # wbuffer.insert(elem[0], elem[1])
            if not wbuffer.insert(elem[0], elem[1]):
                print("ERROR AT " + str(i) + ", " + str(j))

        while not h1.over:
            elem = h1.get_next_pair()
            if i == ceil(log2(n)) -1:
                print(elem)
            # wbuffer.insert(elem[0], elem[1])
            if not wbuffer.insert(elem[0], elem[1]):
                print("ERROR AT " + str(i) + ", " + str(j))

        while not h2.over:
            elem = h2.get_next_pair()
            if i == ceil(log2(n)) -1:
                print(elem)
            # wbuffer.insert(elem[0], elem[1])
            if not wbuffer.insert(elem[0], elem[1]):
                print("ERROR AT " + str(i) + ", " + str(j))

        wbuffer.end_round()

In [6]:
# TODO: Final merge in 1 file
# test:
final_dict = {}
dir = dir2

for name in files_to_merge:
    filename = dir + name.split("/")[-1]
    with open(filename, "rb") as f:
        dict = pickle.load(f)
    final_dict.update(dict)

### **tf-idf**

In [7]:
# compute tf-idf
idfs = {} # muy grande, tal vez pasarlo a mem secundaria
for key, postings_list in final_dict.items():
    dfi = len(postings_list)
    idfs[key] = log2(doc_count / dfi)
    for i in range(len(postings_list)):
        # print(postings_list[i])
        docID, count = postings_list[i]
        tf = log2(1 + count)
        idf = idfs[key]
        tfidf = tf * idf
        postings_list[i] = (docID, tfidf)
with open(final + index_filename, "wb") as f:
    pickle.dump(final_dict, f)

In [8]:
# calcular las normas de cada documento
norms = {doc : 0 for doc in files}
for term, postings_list in final_dict.items():
    for docid, tfidf in postings_list:
        norms[docid] += (tfidf ** 2)
for term in norms:
    norms[term] = np.sqrt(norms[term])

### **Query**

In [9]:
importlib.reload(query_runner)
qr = query_runner.QueryRunner(final + index_filename, files, idfs, norms, stoplist)
query = "when there's dirt beneath the dirt"
qr.make_query(query)

[('lyrics/dance_little_liar.txt', np.float64(0.3360896328029296)),
 ('lyrics/when_the_sun_goes_down.txt', np.float64(0.024473499789036634)),
 ('lyrics/4_out_of_5.txt', np.float64(0.0)),
 ('lyrics/505.txt', np.float64(0.0)),
 ('lyrics/arabella.txt', np.float64(0.0)),
 ('lyrics/batphone.txt', np.float64(0.0)),
 ('lyrics/cornerstone.txt', np.float64(0.0)),
 ('lyrics/no1_party_anthem.txt', np.float64(0.0)),
 ('lyrics/one_for_the_road.txt', np.float64(0.0))]